# Data Analysis: CE Survey Data and Purchase Mode

Starting in 2017, the publicly available CE data began including a column indicating whether the purchase was made online, in-person or through other methods. In this post, I want to compile the results, add descriptions, and summarize with very simple proportions.

The PUMD from the CE Interview can be found <a href="https://www.bls.gov/cex/pumd_data.htm">here</a> and I downloaded the 2017 data as SAS files. All available formats include data for the calendar quarters 2017Q1 - 2018Q1, but 2017Q1 will not be needed since the purchase mode data was not collected then.

#### Create Environment

We will need Pandas and Numpy, but we also need Plotly libraries for creating interactive data tables. Plotting with Plotly can be quite involved, but I personally enjoy it as a plotting module. Lastly, have started printing the version number of the libraries since I ran into trouble reproducing earlier posts due to updates.

In [1]:
import pandas as pd
import numpy as np
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

print('Plotly Version: ' + plotly.__version__)
print('Pandas Version: ' + pd.__version__)
print('Numpy Version: ' + np.__version__)

Plotly Version: 3.6.1
Pandas Version: 0.24.1
Numpy Version: 1.15.4


### Part 1: Compile CE Data Files

From the looks of the <a href="https://www.bls.gov/cex/pumd_doc.htm">PUMD documentation</a> and the spreadsheet <a href="https://www.bls.gov/cex/pumd/ce_pumd_interview_diary_dictionary.xlsx">Dictionary for Interview and Diary Survey</a>, questions about purchase mode were included in 8 CE expenditure sections--I will only review 5 of the 8 in this post.

#### Importing CE Expenditure Section Files
Below is a function <code>import_ce_exp</code> that imports CE expenditure files. The function takes exactly one argument <code>exp_file</code> that is represents the CE expenditure file name. Using <code>exp_file</code>, the file path of the expenditure table is built and stored as <code>exp_file_name</code>. Lastly, the expenditure table is imported using the Pandas library.

In [2]:
def import_ce_exp(exp_file):
    exp_file_name = "C:/Users/Barbieri/Desktop/Research and Education/CE Micro Data/expn17/" + str(exp_file) + "17.sas7bdat"
    
    exp_df = pd.read_sas(exp_file_name, encoding='latin-1')
    
    return exp_df

Below we use the function <code>import_ce_exp</code> to import each CE expenditure table.

We create a list called <code>exp_names</code> where each element is the name of a CE expenditure file. This list will be used throughout the notebook. Then we create a dictionary called <code>dict_df_exp</code> where each element in the dictionary will point to a CE expenditure file, imported as a Pandas DataFrame. Each element in the list <code>exp_names</code> is passed both as the argument for the <code>import_ce_exp</code> function, as well as the key name for each DataFrame in the dictionary <code>dict_df_exp</code>.

In [3]:
exp_names = ['apb','mis','eqb','ovb','sub']
dict_exp_df = {}

for ce_exp in exp_names:
    dict_exp_df[ce_exp] = import_ce_exp(ce_exp)

dict_exp_df.keys()

dict_keys(['apb', 'mis', 'eqb', 'ovb', 'sub'])

### Part 2: Data Manipulation

Next, we need to reorganize the data for various reasons. First, the column that refers to purchase mode (our column of interest) and the column that refers to the item category purchased are named differently on each expenditure file. Second, if our goal is to compile all of the expenditure tables into one DataFrame for analysis, then we can drop most of the variables as they do not overlap. Third, since the majority of respondents are not asked about purchase mode, there is no point in keeping values where purchase mode is blank. Lastly, item categories and the purchase mode are meaningless as numeric codes. We will import a concordance I created between the item category codes and their descriptions using the CE data documentation, as well as hard-code descriptions of purchase mode codes.

To Summarize:
1. Rename the columns that represent the expenditure category and the purchase mode.
2. Drop rows/instances where purchase mode is blank.
3. Keep only overlapping columns across expenditure tables.
4. Add descriptions to the CE expenditure item codes and values stored in the purchase mode column.

#### Rename and Keep Columns, Drop Rows

Below we create a function called <code>rename_keep_dropna</code> that takes three arguments, <code>exp_file</code>, <code>rename_purch</code>, and <code>rename_item</code>. The first argument <code>exp_file</code> is the name of the expenditure file as it was stored in the list <code>exp_names</code>. It is used to locate the expenditure file in the dictionary <code>dict_exp_df</code> and is used as a condition for renaming the purchase mode column and item category column. The second and third arguments are the column names that will replace the purchase mode and item category column names, respectively.

After the function is defined, it is called for each element in the list <code>exp_names</code>.

In [4]:
def rename_keep_dropna(exp_file, rename_purch, rename_item):
    
    #1. Rename Purchase Mode and Item Category Columns.
    if exp_file == 'apb':
        dict_exp_df[exp_file].rename(columns={"APBPURCH":str(rename_purch),
                                              "MINAPPLY":str(rename_item)},
                                     inplace=True)
    elif exp_file == 'mis':
        dict_exp_df[exp_file].rename(columns={"MISPURCH":str(rename_purch), 
                                              "MISCCODE":str(rename_item)},
                                     inplace=True)
    elif exp_file == 'eqb':
        dict_exp_df[exp_file].rename(columns={"EQBPURCH":str(rename_purch),
                                              "APPRPRYB":str(rename_item)},
                                     inplace=True)
    elif exp_file == 'ovb':
        dict_exp_df[exp_file].rename(columns={"OVBPURCH":str(rename_purch),
                                              "VEHICYB":str(rename_item)},
                                     inplace=True)
    elif exp_file == 'sub':
        dict_exp_df[exp_file].rename(columns={"SUBPURCH":str(rename_purch),
                                              "S17CODEA":str(rename_item)},
                                     inplace=True)

    #2. Drop rows with missing purchase mode values.
    dict_exp_df[exp_file] = dict_exp_df[exp_file].dropna(axis='index', how='any', subset=[rename_purch])
    
    #3. Keep only overlapping columns.
    dict_exp_df[exp_file] = dict_exp_df[exp_file][['QYEAR', rename_purch, rename_item]]

In [5]:
for ce_exp in exp_names:
    rename_keep_dropna(ce_exp, "PURCH_TYPE", "EXPN_TYPE")

#### Add Descriptions to Item Categories

First, let's import the concordance table and store it in a Pandas DataFrame called <code>item_cat_desc_df</code>. The concordance has a column called <code>File</code> that corresponds to the name of the expenditure table. It is used to subset the concordance to only those item categories we expect in a particular expenditure file.

In [6]:
item_cat_desc_df = pd.read_excel("C:/Users/Barbieri/Desktop/Research and Education/CE Micro Data/Expn_Code_Descriptions.xlsx",
                                  dtype=object)

item_cat_desc_df.head()

,File Name,Variable Name,Item Category,Item Category Description
0,OVB,VEHICYB,100,Automobile
1,OVB,VEHICYB,105,"Automobile, truck, van, miniva"
2,OVB,VEHICYB,110,"Truck, van, minivan, or SUV"
3,OVB,VEHICYB,120,Motor Home
4,OVB,VEHICYB,130,Trailer Type Camper


Now, we create a function called <code>expn_type_desc</code> that subsets the concordance to a defined CE expenditure table and merges the descriptions with the expenditure table on the column <code>EXPN_TYPE</code>. The merge is an inner merge, which means that only values of <code>EXPN_TYPE</code> that are in both the CE expenditure table and the subset item concordance will successfully merge.

Much like the other functions, <code>expn_type_desc</code> is then called for each element in the list <code>exp_names</code>.

In [7]:
def expn_type_desc(exp_file):
    code_list = item_cat_desc_df[item_cat_desc_df['File Name'] == exp_file.upper()]
    
    dict_exp_df[exp_file] = dict_exp_df[exp_file].merge(code_list,
                                                        how='inner',
                                                        left_on='EXPN_TYPE',
                                                        right_on='Item Category')

In [8]:
for ce_exp in exp_names:
    expn_type_desc(ce_exp)

#### Add Descriptions to Purchase Mode

Below is a function <code>purch_type_desc</code> that hard-codes the description for each possible value of the purchase mode column.

After the function is defined, it is applied using the <code>apply</code> attribute. In this case, a new column is created called <code>OTYPE_DESC</code> that represents the converted value of <code>PURCH_TYPE</code>.

In [9]:
def purch_type_desc(purch_type):
    if purch_type == "1":
        purch_type_desc = "Online"
    elif purch_type == "2":
        purch_type_desc = "In-Person"
    elif purch_type == "3":
        purch_type_desc = "Other"
    else:
        purch_type_desc = purch_type

    return purch_type_desc

In [10]:
for ce_exp in exp_names:
    dict_exp_df[ce_exp]['OTYPE_DESC'] = dict_exp_df[ce_exp]['PURCH_TYPE'].apply(purch_type_desc)

#### Combine All Expenditure Files

Now we can combine all of the expenditure files into <i>one</i> DataFrame called <code>ce_exp_df</code> which will be used in the analysis.

In [11]:
for elem_num, ce_exp in enumerate(exp_names):
    if elem_num == 0:
        ce_exp_df = dict_exp_df[ce_exp]
    else:
        ce_exp_df = pd.concat([ce_exp_df, dict_exp_df[ce_exp]], ignore_index=True)

I ignored diagnostics in the final version of this notebook, but let's run one summary command to give us a sense of the number of observations by expenditure table.

In [12]:
ce_exp_df['File Name'].value_counts()

OVB    643
MIS    137
APB    121
SUB     35
EQB     10
Name: File Name, dtype: int64

### Part 3: Data Analysis and Visualization

Instead of plainly printing the proportions of reported purchase methods grouped by expenditure category, we are going to use Plotly library to display the proportions on a table, fit with a dropdown menu where users can select the item category.

Our goal is to populate a table with the portions of reported purchase modes by item category. To do so, Plotly will require that each item category's proportions be a <i>separate graph object</i>. We will then use Plotly's menu options to link each graph object to a button in a dropdown menu. That way, when a user selects the category "Arts & Crafts Supplies" from the dropdown menu, the purchase mode proportions for Arts & Craft Supplies populates the table.

#### Calculate Purchase Mode Proportions by Item Category

We are going to use a for-loop to calculate the proportions of purchase modes for each item category in the DataFrame <code>ce_exp_df</code> and assign it to a Plotly graph object using Plotly's <code>Table</code> function. The proportion of purchase modes reported for each item category is stored in the Series <code>purch_prop</code>. Each table object will be stored in the list <code>table_traces</code>.

In [13]:
table_traces = []

ce_exp_df_gb = ce_exp_df.groupby(['Item Category Description'])

for group, purch in ce_exp_df_gb:    
    purch_prop = purch['OTYPE_DESC'].value_counts(normalize=True)
    
    if len(purch_prop.index) == 1:
        header_values=[str(purch_prop.index[0])]
    
    elif len(purch_prop.index) == 2:
        header_values=[str(purch_prop.index[0]),
                       str(purch_prop.index[1])]
    
    elif len(purch_prop.index) == 3:
        header_values=[str(purch_prop.index[0]),
                       str(purch_prop.index[1]),
                       str(purch_prop.index[2])]

    table = go.Table(header=dict(values=header_values),
                     cells=dict(values=purch_prop))

    table_traces.append(table)

#### Create Buttons for Dropdown Menu

The length of the list <code>table_traces</code> will tell us how many different item categories there are and therefore, how many buttons we will need in the dropdown menu. Each button will contain an attribute called <code>visible</code> to tell Plotly which item categories to make visible when a particular item from the dropdown menu is selected. In our case, we only want one trace (item category) visible at a time.

Below we create a list called <code>cat_visual_attrib_bool</code> that we will use to populate the attribute <code>visible</code> for each button. The list will have a total number of elements equal to the number of item categories. Each element will contain a series of True and False values, in particular, one True value and the remaining False.

In [14]:
num_groups = len(table_traces)
array_bool_false = np.full((1, num_groups), False)
cat_visual_attrib_bool = []

for i in range(0, num_groups):
    list_bool = array_bool_false[0].tolist()
    list_bool[i] = True
    
    cat_visual_attrib_bool.append(list_bool)

Next, we define each button and their attributes. For each item category, we label the button using the <code>Item Category Description</code> column from the <code>ce_exp_df_gb</code> DataFrame, assign the method "update" to indicate that the table should be regenerated with the requested data, and assign one of the elements from <code>cat_visual_attrib_bool</code> to the <code>visible</code> attribute.

In [15]:
button_attribs = []

for index, purch_prop in enumerate(ce_exp_df_gb):
    attributes = dict(label=purch_prop[0],
                          method='update',
                          args=[dict(visible=cat_visual_attrib_bool[index])])
    button_attribs.append(attributes)

Lastly, we define the dropdown menu by setting the <code>type</code> attribute to 'dropdown'. Each button available in the dropdown menu is defined by the button attributes stored in the list <code>button_attribs</code>, created above. I use the values <code>x</code> and <code>y</code> to define the location of the dropdown menu.

In [16]:
updatemenus = list([dict(type='dropdown',
                         active=-1,
                         buttons=button_attribs,
                         x=0.65,
                         y=0.80)
                   ])

#### Generate the Table with Dropdown Menu

Finally, we assign our menu options <code>updatemenus</code> to the <code>updatemenus</code> attribute in a dictionary called <code>layout</code>. A dictionary called <code>figure</code> contains one element for the data and another for the figure layout. Lastly, running <code>iplot</code> by passing the dictionary <code>figure</code> will produce the table with the dropdown menu. Feel free to explore!

In [17]:
layout = dict(title='Proportion of Purchase Mode by Item Category',
              updatemenus=updatemenus)

figure = dict(data=table_traces, layout=layout)

iplot(figure)